<div class="alert alert-success">
    <h1>Session 1: Basic PyTorch Tutorial</h1>
    <h3 align='center'>Computational Intelligence (4032-01)</h3>
    <h5 align='center'>Instructor: Dr.Samane Hosseini</h5>
    <h5 align='center'>TA: Arash Azhand</h5>
</div>

# PyTorch Basics Tutorial

This tutorial covers:
1. Tensors in PyTorch
2. Automatic Differentiation
3. Neural Network Basics
4. Dataset and Dataloader
5. Training a Simple Neural Network
6. Evaluating the Model
7. Saving and Loading Models
8. GPU Acceleration
9. Custom Datasets
Conclusion

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import matplotlib.pyplot as plt

# 1. Tensors in PyTorch
**Tensors are the fundamental data structure in PyTorch, similar to NumPy arrays but with GPU support.**

In [2]:
tensor_a = torch.tensor([1, 2, 3], dtype=torch.float32)  # Creating a tensor
print("Tensor A:", tensor_a)

Tensor A: tensor([1., 2., 3.])


In [3]:
# Operations on tensors
tensor_b = torch.tensor([4, 5, 6], dtype=torch.float32)
tensor_sum = tensor_a + tensor_b  # Element-wise addition
print("Tensor Sum:", tensor_sum)

Tensor Sum: tensor([5., 7., 9.])


In [4]:
tensor_c = torch.rand((3, 3))  # Random tensor
print("Random Tensor:", tensor_c)

Random Tensor: tensor([[0.8060, 0.9662, 0.3760],
        [0.2699, 0.2057, 0.2360],
        [0.4044, 0.3408, 0.4271]])


In [5]:
# Reshaping tensors
tensor_reshaped = tensor_c.view(1, 9)  # Reshape to 1 row, 9 columns
print("Reshaped Tensor:", tensor_reshaped)

Reshaped Tensor: tensor([[0.8060, 0.9662, 0.3760, 0.2699, 0.2057, 0.2360, 0.4044, 0.3408, 0.4271]])


In [6]:
# Tensor operations like matrix multiplication
tensor_d = torch.rand((3, 3))
tensor_e = torch.rand((3, 3))
tensor_matmul = torch.matmul(tensor_d, tensor_e)
print("Matrix Multiplication Result:", tensor_matmul)

Matrix Multiplication Result: tensor([[0.2795, 0.7298, 0.9133],
        [0.4123, 0.7092, 0.6409],
        [0.7564, 1.2861, 1.2675]])


# 2. Automatic Differentiation
**PyTorch uses automatic differentiation to compute gradients, which is essential for training neural networks**

In [7]:
x = torch.randn(3, requires_grad=True)  # Track computation for automatic differentiation
y = x * 10
y = y.sum()
y.backward()  # Compute gradients
print("Gradients:", x.grad)

Gradients: tensor([10., 10., 10.])


Here, `x.grad` stores the gradient of `y` with respect to `x`.

In [8]:
# Clearing gradients (important in training loops)
x.grad.zero_()

tensor([0., 0., 0.])

# 3. Neural Network Basics
**PyTorch provides the `nn.Module` class to define neural networks.**

In [9]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Fully connected layer with 2 inputs and 4 outputs
        self.fc2 = nn.Linear(4, 1)  # Fully connected layer with 4 inputs and 1 output
    
    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation function
        x = self.fc2(x)
        return x

model = SimpleNN()
print("Neural Network Model:", model)

Neural Network Model: SimpleNN(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)


# 4. Dataset and Dataloader
**PyTorch provides `Dataset` and `DataLoader` classes to handle data loading and batching.**

In [10]:
X_train = torch.rand((100, 2))  # 100 samples, 2 features
y_train = torch.rand((100, 1))  # 100 labels
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [11]:
# Custom Dataset Example
# NOTE: We mostly use this when our data is not in the form of tensors (eg., images, text, speech, etc.).
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

custom_dataset = CustomDataset(X_train, y_train)
custom_loader = DataLoader(custom_dataset, batch_size=10, shuffle=True)

# 5. Training a Simple Neural Network
**Training involves forward pass, loss computation, backward pass, and optimization.**

In [12]:
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer

In [13]:
epochs = 10
for epoch in range(epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.3090
Epoch 2, Loss: 0.1247
Epoch 3, Loss: 0.0934
Epoch 4, Loss: 0.0870
Epoch 5, Loss: 0.0891
Epoch 6, Loss: 0.0857
Epoch 7, Loss: 0.0841
Epoch 8, Loss: 0.0856
Epoch 9, Loss: 0.0824
Epoch 10, Loss: 0.0823


# 6. Evaluating the Model
**Evaluation involves running the model on validation/test data without updating weights.**

In [14]:
def evaluate(model, data_loader):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f"Evaluation Loss: {total_loss / len(data_loader):.4f}")

evaluate(model, train_loader)

Evaluation Loss: 0.0819


# 7. Saving and Loading Models
**Save model state**

In [15]:
torch.save(model.state_dict(), "simple_nn.pth")
print("Model saved.")

Model saved.


In [16]:
# Load model state
loaded_model = SimpleNN()
loaded_model.load_state_dict(torch.load("simple_nn.pth"))
print("Model loaded successfully.")

Model loaded successfully.


C:\Users\arash\AppData\Local\Temp\ipykernel_40772\1852773834.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("simple_nn.pth"))


# 8. GPU Acceleration
**PyTorch allows you to move tensors and models to GPU for faster computation.**

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [18]:
# Move model and data to GPU
model.to(device)
X_train_gpu = X_train.to(device)
y_train_gpu = y_train.to(device)

# 9. Custom Datasets and Data Augmentation
**PyTorch allows you to create custom datasets and apply data augmentation.**

In [19]:
from torchvision import transforms

# Example of data augmentation using torchvision transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize data
])

# Conclusion
### PyTorch is a powerful and flexible framework for deep learning. This tutorial covers the basics, but there's much more to explore, including advanced architectures, distributed training, and more.